In [2]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
df = pd.read_csv('training_base.csv')

In [4]:
df.shape

(3576, 10)

In [5]:
def remove_space(text):
    text = text.strip()
    text = re.sub("\s\s+", " ", text)
    return text
    
df['tweet_compound'] = df.tweet_compound.map(remove_space)
df['tweet'] = df.tweet.map(remove_space)

In [6]:
df.to_csv('training_base_new.csv')

In [88]:
df1 = pd.read_csv('training_base_new.csv')

,Unnamed: 0,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,
0,1375,1375,1375,1375,1375,1375,1375,1375,1375
2,1621,1621,1621,1621,1621,1621,1621,1621,1621
3,227,227,227,227,227,227,227,227,227
4,353,353,353,353,353,353,353,353,353


In [8]:
train, test = train_test_split(df1, test_size = 0.2, random_state=10)

In [89]:
train.groupby('intent').count()

,Unnamed: 0,Unnamed: 0.1,username,tweet,tweet_compound,topic,neg,neu,pos,compound
intent,,,,,,,,,,
0,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111
2,1280,1280,1280,1280,1280,1280,1280,1280,1280,1280
3,182,182,182,182,182,182,182,182,182,182
4,287,287,287,287,287,287,287,287,287,287


In [9]:
stop = stopwords.words('english')

# Intent analysis

## Naive Bayes

In [41]:
X_tr1 = train['tweet']
y_tr1 = train[['intent']]

In [42]:
# creating bag of words model 
cv = CountVectorizer(max_features=1100) 
  
X1 = cv.fit_transform(X_tr1).toarray() 
y1 = y_tr1.iloc[:, 0].values 

# fitting naive bayes to the training set 
from sklearn.naive_bayes import MultinomialNB 

classifier = MultinomialNB();
classifier.fit(X1, y1) 

MultinomialNB()

In [43]:
test_nb1 = test.copy()
X_test1 = test['tweet']
X_t1_cv = cv.fit_transform(X_test1).toarray() 

In [44]:
y_pred1 = classifier.predict(X_t1_cv) 

In [45]:
test_nb1['pred'] =  classifier.predict(X_t1_cv)

## Round 2 Naive Bayes

In [46]:
X_tr2 = train['tweet']
y_tr2 = train[['intent']]

In [47]:
# creating bag of words model 
cv2 = CountVectorizer(max_features=1100) 
  
X2 = cv2.fit_transform(X_tr2).toarray() 
y2 = y_tr1.iloc[:, 0].values 

classifier2 = MultinomialNB();
classifier2.fit(X2, y2) 

MultinomialNB()

In [48]:
test_nb2 = test.copy()
X_test2 = test['tweet']
X_t2_cv = cv2.fit_transform(X_test2).toarray() 

In [49]:
y_pred2 = classifier2.predict(X_t2_cv) 

In [50]:
test_nb2['pred'] =  classifier2.predict(X_t2_cv)

In [51]:
accuracy2 = accuracy_score(test['intent'], y_pred2)
print("Accuracy: %.2f%%" % (accuracy2 * 100.0))

Accuracy: 9.08%


In [52]:
confusion_matrix(test['intent'], test_nb2['pred'])

array([[ 15,   5, 155,  89],
       [ 22,   9, 192, 118],
       [  2,   2,  26,  15],
       [  7,   5,  39,  15]])

Very poor performance - do not pursue any further

## With XGBoost
### No tuning

In [149]:
X_tr3 = train['tweet']
y_tr3 = train[['intent']]

In [171]:
# creating bag of words model 
cv3 = CountVectorizer(stop_words = stop,
                      max_features=880) 
  
X3 = cv3.fit_transform(train['tweet']).toarray() 
y3 = y_tr3.iloc[:, 0].values 

# fitting XGB to the training set 
gbm3 = xgb.XGBClassifier(max_depth=5,
                         learning_rate = 0.05,
                         objective = 'multi:softmax',
                         num_class = 4)

In [172]:
gbm3.fit(X3,y3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_class=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [173]:
test_gbm3 = test.copy()
X_test3 = test['tweet']
X_t3_cv = cv3.fit_transform(X_test3).toarray() 
y_pred3 = gbm3.predict(X_t3_cv)
gbm_pred3 = [round(value) for value in y_pred3]
test_gbm3['pred'] =  gbm_pred3

In [174]:
accuracy3 = accuracy_score(test['intent'], gbm_pred3)
print("Accuracy: %.2f%%" % (accuracy3 * 100.0))

Accuracy: 49.02%


In [175]:
confusion_matrix(test['intent'], test_gbm3['pred'])

array([[ 91, 150,   5,  18],
       [ 69, 257,   3,  12],
       [ 11,  31,   2,   1],
       [  8,  57,   0,   1]])

In [176]:
print(classification_report(test['intent'], test_gbm3['pred']))

              precision    recall  f1-score   support

           0       0.51      0.34      0.41       264
           2       0.52      0.75      0.61       341
           3       0.20      0.04      0.07        45
           4       0.03      0.02      0.02        66

    accuracy                           0.49       716
   macro avg       0.31      0.29      0.28       716
weighted avg       0.45      0.49      0.45       716



In [165]:
test_gbm3.groupby('pred').count()

,Unnamed: 0,Unnamed: 0.1,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound
pred,,,,,,,,,,,
0,179,179,179,179,179,179,179,179,179,179,179
2,495,495,495,495,495,495,495,495,495,495,495
3,10,10,10,10,10,10,10,10,10,10,10
4,32,32,32,32,32,32,32,32,32,32,32


## Tuned parameters
### Need to update

In [30]:
X_tr4 = train['tweet']
y_tr4 = train[['intent']]

In [31]:
# creating bag of words model 
cv4 = CountVectorizer(stop_words = stop,
                      max_features=880) 
  
X4 = cv4.fit_transform(X_tr4).toarray() 
y4 = y_tr4.iloc[:, 0].values 

# fitting XGB to the training set 
gbm4 = xgb.XGBClassifier(max_depth = 7,
                         learning_rate = 0.05,
                         subsample= 0.9,
                         colsample_bytree = 0.8,
                         objective= 'multi:softmax',
                         num_class = 4)

In [32]:
gbm4.fit(X4,y4)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_class=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
test_gbm4 = test.copy()
X_test4 = test_gbm4['tweet']
X_t4_cv = cv4.fit_transform(X_test4).toarray() 
y_pred4 = gbm4.predict(X_t4_cv)
gbm_pred4 = [round(value) for value in y_pred4]
test_gbm4['pred'] =  gbm_pred4

In [34]:
accuracy4 = accuracy_score(test['intent'], gbm_pred4)
print("Accuracy: %.2f%%" % (accuracy4 * 100.0))

Accuracy: 50.00%


In [35]:
confusion_matrix(test['intent'], test_gbm4['pred'])

array([[106, 132,   5,  21],
       [ 67, 248,   4,  22],
       [ 10,  29,   3,   3],
       [ 13,  52,   0,   1]])

In [36]:
print(classification_report(test['intent'], test_gbm4['pred']))

              precision    recall  f1-score   support

           0       0.54      0.40      0.46       264
           2       0.54      0.73      0.62       341
           3       0.25      0.07      0.11        45
           4       0.02      0.02      0.02        66

    accuracy                           0.50       716
   macro avg       0.34      0.30      0.30       716
weighted avg       0.47      0.50      0.47       716



In [179]:
import pickle
# save the model to disk
filename1 = 'intentxgb_model.sav'
pickle.dump(gbm4, open(filename1, 'wb'))

Worse than the first model, as expected. Need to redo the gridsearch given now have 4 classes. 

### XGB with ROS

In [93]:
## RF with Oversampling
# Now add some random oversampling of the minority classes
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# ros = RandomOverSampler(random_state=0)
# X_resampled, y_resampled = ros.fit_sample(train[['tweet']], train['intent'])

# from collections import Counter
# # Yay, balanced classes!
# Counter(y_resampled)

In [127]:
# define resampling
strategyo = {3:500, 4:500}
strategyu = {0:500, 2:500}
over = RandomOverSampler(sampling_strategy=strategyo)
under = RandomUnderSampler(sampling_strategy=strategyu)
# define pipeline
pipeline = Pipeline(steps=[('o', over), ('u', under)])

In [128]:
# fit and apply the pipeline
X_resampled, y_resampled = pipeline.fit_resample(train[['tweet']], train['intent'])

In [129]:
y_resampled.value_counts()

4    500
3    500
2    500
0    500
Name: intent, dtype: int64

In [130]:
X_resampled_ser = X_resampled['tweet']

In [131]:
# creating bag of words model 
cv6 = CountVectorizer(stop_words = stop,
                      max_features=880) 
  
X6 = cv6.fit_transform(X_resampled_ser).toarray() 
y6 = y_resampled.to_numpy()

In [137]:
# fitting XGB to the training set 
gbm6 = xgb.XGBClassifier(max_depth = 7,
                         learning_rate = 0.05,
                         subsample= 0.9,
                         colsample_bytree = 0.8,
                         objective= 'multi:softmax',
                         num_class = 4)

gbm6.fit(X6,y6)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_class=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [138]:
test_gbm6 = test.copy()
X_test6 = test_gbm4['tweet']
X_t6_cv = cv6.fit_transform(X_test6).toarray() 
y_pred6 = gbm6.predict(X_t6_cv)
gbm_pred6 = [round(value) for value in y_pred6]
test_gbm6['pred'] =  gbm_pred6

In [139]:
accuracy6 = accuracy_score(test['intent'], gbm_pred6)
print("Accuracy: %.2f%%" % (accuracy6 * 100.0))

Accuracy: 21.93%


In [140]:
confusion_matrix(test['intent'], test_gbm6['pred'])

array([[ 51,  19,  21, 173],
       [ 21,  52,  27, 241],
       [  4,   2,   7,  32],
       [  4,   4,  11,  47]])

In [141]:
print(classification_report(test['intent'], test_gbm6['pred']))

              precision    recall  f1-score   support

           0       0.64      0.19      0.30       264
           2       0.68      0.15      0.25       341
           3       0.11      0.16      0.13        45
           4       0.10      0.71      0.17        66

    accuracy                           0.22       716
   macro avg       0.38      0.30      0.21       716
weighted avg       0.57      0.22      0.25       716



## Random forest

In [37]:
#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X4,y4)

X_test4_rf = cv4.fit_transform(test['tweet']).toarray() 
y_pred4_rf = clf.predict(X_test4_rf)

In [38]:
print("Accuracy:",accuracy_score(test['intent'], y_pred4_rf))

Accuracy: 0.40502793296089384


In [39]:
confusion_matrix(test['intent'], y_pred4_rf)

array([[131,  85,  13,  35],
       [148, 133,  15,  45],
       [ 16,  13,   7,   9],
       [ 15,  29,   3,  19]])

In [40]:
print(classification_report(test['intent'], y_pred4_rf))

              precision    recall  f1-score   support

           0       0.42      0.50      0.46       264
           2       0.51      0.39      0.44       341
           3       0.18      0.16      0.17        45
           4       0.18      0.29      0.22        66

    accuracy                           0.41       716
   macro avg       0.32      0.33      0.32       716
weighted avg       0.43      0.41      0.41       716



Better than previous models in terms of predicting class 3, which is important. However, worse performance overall

In [59]:
## RF with Oversampling
# Now add some random oversampling of the minority classes
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(train[['tweet']], train['intent'])

from collections import Counter
# Yay, balanced classes!
Counter(y_resampled)

Counter({0: 1280, 2: 1280, 3: 1280, 4: 1280})

In [63]:
X_resampled_ser = X_resampled['tweet']

In [65]:
y_resampled

0       0
1       0
2       2
3       0
4       3
       ..
5115    4
5116    4
5117    4
5118    4
5119    4
Name: intent, Length: 5120, dtype: int64

In [68]:
# creating bag of words model 
cv6 = CountVectorizer(stop_words = stop,
                      max_features=880) 
  
X6 = cv6.fit_transform(X_resampled_ser).toarray() 
y6 = y_resampled.to_numpy()

In [69]:
#Create a Gaussian Classifier
clf6 = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf6.fit(X6,y6)

X_test6_rf = cv6.fit_transform(test['tweet']).toarray() 
y_pred6_rf = clf6.predict(X_test6_rf)

In [70]:
print("Accuracy:",accuracy_score(test['intent'], y_pred6_rf))

Accuracy: 0.30307262569832405


In [71]:
confusion_matrix(test['intent'], y_pred6_rf)

array([[ 99,  51,  37,  77],
       [105,  87,  72,  77],
       [ 10,  14,  11,  10],
       [ 15,  17,  14,  20]])

In [72]:
print(classification_report(test['intent'], y_pred6_rf))

              precision    recall  f1-score   support

           0       0.43      0.38      0.40       264
           2       0.51      0.26      0.34       341
           3       0.08      0.24      0.12        45
           4       0.11      0.30      0.16        66

    accuracy                           0.30       716
   macro avg       0.28      0.29      0.26       716
weighted avg       0.42      0.30      0.33       716

